In [15]:
from mangaki.algo.dataset import Dataset
from mangaki.utils.values import rating_values

queryset = Rating.objects.all()  #.exclude(choice__in=['willsee', 'wontsee']) # Security if the dataset is public
triplets = queryset.values_list('user_id', 'work_id', 'choice')
titles = Work.objects.order_by('id').values_list('id', 'title')
categories = Work.objects.order_by('id').values_list('id', 'category__slug')
dataset = Dataset()

In [18]:
anonymized = dataset.make_anonymous_data(triplets, convert=lambda choice: rating_values[choice], ordered=True)

In [19]:
dataset.anonymized

AnonymizedData(X=array([[1511, 8376],
       [1016,  469],
       [1194, 1895],
       ...,
       [  35,   33],
       [1088,  842],
       [ 448,  623]]), y=array([ 0.5,  0.5, -0.5, ...,  0.5,  0.1,  0.5]), y_text=array(['willsee', 'willsee', 'wontsee', ..., 'willsee', 'neutral',
       'willsee'], dtype='<U8'), nb_users=2210, nb_works=11946)

In [23]:
import pandas as pd

df = pd.DataFrame(np.column_stack((dataset.anonymized.X, dataset.anonymized.y)), columns=('user', 'item', 'rating'))
df.head()

,user,item,rating
0,1511.0,8376.0,0.5
1,1016.0,469.0,0.5
2,1194.0,1895.0,-0.5
3,1640.0,157.0,0.5
4,1503.0,136.0,2.0


In [25]:
df.to_csv('/Users/jilljenn/code/vae/data/mangaki/data.csv', index=False)

In [27]:
df.max() + 1

user       2210.0
item      11946.0
rating        5.0
dtype: float64

In [3]:
dataset.anonymized.y

array([-2. ,  2. ,  2. , ...,  2. ,  0.5, -2. ])

In [4]:
dataset.titles = dict(titles)
dataset.categories = dict(categories)

In [5]:
dataset.save_csv(suffix='-ordered')

`/Users/jilljenn/code/mangaki/mangaki/data/ratings-ordered.csv` or `/Users/jilljenn/code/mangaki/mangaki/data/works-ordered.csv` already exists. Overwrite? [y/n] y


In [6]:
dataset.save('tsne.pickle')

In [7]:
import pandas as pd

In [8]:
import numpy as np

arr = np.random.random((4, 2))

In [9]:
pd.DataFrame(arr).to_csv

<bound method DataFrame.to_csv of           0         1
0  0.643767  0.621476
1  0.391406  0.729256
2  0.516019  0.347584
3  0.693055  0.426127>

In [10]:
import os

os.system('cat test.csv')

256

In [19]:
pd.DataFrame(arr).to_csv('test.csv', header=False, index=False)

In [24]:
np.column_stack((arr, np.ones(4)))

array([[ 0.23661292,  0.37464382,  1.        ],
       [ 0.23299154,  0.54219249,  1.        ],
       [ 0.84781838,  0.97769781,  1.        ],
       [ 0.7231793 ,  0.01685298,  1.        ]])

In [28]:
np.hstack((arr, np.ones(4).reshape(4, 1)))

array([[ 0.23661292,  0.37464382,  1.        ],
       [ 0.23299154,  0.54219249,  1.        ],
       [ 0.84781838,  0.97769781,  1.        ],
       [ 0.7231793 ,  0.01685298,  1.        ]])